# Naive Bayes

朴素贝叶斯法是基于贝叶斯定理与特征条件独立假设的分类方法，选择后验概率最大的类别作为预测类别。

## 算法：
- (1)使用极大似然估计先验概率与类条件概率，并做拉普拉斯平滑处理：
$$P_{\alpha}(Y=c_k)=\frac{\sum_{i=1}^{N}I(y_i=c_k)+\alpha}{N+K\alpha}$$
其中$N$为总样本数，$\alpha$为拉普拉斯平滑的参数，$K$为类数。
$$P_\alpha(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^{N}I(x_i^{(j)}=a_{jl},j_i=c_k)+\alpha}{\sum_{i=1}^{N}I(y_i=c_k)+S_j\alpha}$$
为第j个特征的条件概率，其中$S_j$为第j个特征可能取值的个数。


- (2)使用Bayes公式计算后验概率:
$$P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k)$$


- (3)后验概率最大的类别作为预测类别

**注：** 本算法为离散数据的Bayes算法，若特征数据为连续数值型，可以假设数据服从高斯分布，使用数据估计该特征的均值与方差（极大似然估计）或者将数据离散化处理。

In [1]:
import numpy as np

In [23]:
class NaiveBayes():

    def __init__(self, alpha=1):
        self.alpha = alpha
        self.class_prior_ = None #类的先验概率
        self.probabilities_ = None  #类的条件概率
        self.post_proba_ = None  #后验概率
        
    def get_class_prior(self, y):
        '''
        输入为训练集的标签,为一维数组，得到类的先验概率密度，以字典{类：先验概率}形式储存数据
        e.g. y = np.array(["类1","类1","类2","类2"])
        get_class_prior(y) = {'类1': 0.5, '类2': 0.5}
        '''
        labels = list(y)
        class_prior = {}
        for i in set(y):
            class_prior[i] = (labels.count(i) + self.alpha)/(len(labels) + len(set(y)) * self.alpha)# 用频率估计概率
        return class_prior
        
    def get_cond_proba(self, X, y):
        '''
        输入参数X为二维数组，标签y为一维数组，计算类的条件概率。以嵌套字典形式存储数据
        e.g.
        a = np.array([[1,2,2],[2,3,2],[1,3,1],[2,2,2]]),b = np.array(["类1","类1","类2","类2"])
        get_cond_proba(a,b):{'类1': {0: {1: 0.5, 2: 0.5}, 1: {2: 0.5, 3: 0.5}, 2: {2: 1.0}},
                             '类2': {0: {1: 0.5, 2: 0.5}, 1: {2: 0.5, 3: 0.5}, 2: {1: 0.5, 2: 0.5}}}
        最外层为类标签，第2层为第i个特征，第3层为第i个特征的第j个取值，最后的值为这个取值的联合概率。
        
        '''
        probabilities = {}
        for i in set(y):
            feature_dict = {}
            X_class = X[y == i]  #取第i个类的那些数据
            for j, values in enumerate(X_class.T):#这里应该要取转置，用来得到某一类别的某一特征上的所有数据
                values_dict = {}
                for k in set(values):# 计算某一特征上的所有取值的概率（用频率估计概率）
                    values_dict[k] = (list(values).count(k) + self.alpha)/(len(values) + len(set(values)) * self.alpha)
                feature_dict[j] = values_dict
            probabilities[i]=feature_dict
        return probabilities
    
    def compute_proba(self,X_test):
        '''
        计算给定测试数据的后验概率
        e.g. compute_proba([2,3,2])
            输出：{'类1': 0.125, '类2': 0.0625}
        '''
        post_proba = {}
        sum_prob = 0
        for i in self.class_prior_.keys():#先对类别标签进行循环
            proba = self.class_prior_[i] #先验概率
            for j in self.probabilities_[i].keys():
                proba *= self.probabilities_[i][j].get(X_test[j],0)#先验概率*X_test第一个特征对应取值的概率*……*第n个特征的对应取值概率
            post_proba[i] = proba
            sum_prob += proba
        for i in post_proba.keys():# 计算概率
            post_proba[i] = post_proba[i]/sum_prob
        return post_proba
    
    def fit(self, X, y):
        '''
        X为二维特征矩阵，y为一维标签向量
        '''
        self.class_prior_ = self.get_class_prior(y)
        self.probabilities_ = self.get_cond_proba(X,y)
        
        
    def predict(self,X_test):
        '''
        输入为一维特征向量，输出预测标签
        '''
        self.post_proba_ = self.compute_proba(X_test)
        return max(self.post_proba_, key=self.post_proba_.get)#选取后验概率最大的类别


## 测试

In [24]:
a = np.array([[1,2,2],[2,3,2],[1,3,1],[2,2,2]])
b = np.array(["类1","类1","类2","类2"])

In [25]:
clf=NaiveBayes()

In [26]:
clf.fit(a,b)

In [52]:
clf.predict([2,3,2])

'类1'

In [53]:
clf.post_proba_

{'类1': 0.6666666666666666, '类2': 0.3333333333333333}

In [39]:
from sklearn.naive_bayes import CategoricalNB

In [1]:
clf2 = CategoricalNB()
clf2.fit(a,b)

NameError: name 'CategoricalNB' is not defined

In [48]:
clf2.predict(np.array([[2,2,2]]))

array(['类1'], dtype='<U2')

In [49]:
clf2.predict_proba(np.array([[2,2,2]]))

array([[0.6, 0.4]])

## 垃圾邮件分类实战
